In [1]:
import pandas as pd
import functools
import numpy as np
from tqdm import tqdm
import os
from multiprocessing.pool import Pool
from scipy import stats
import seaborn as sns


In [2]:
df_item = pd.read_csv("/media/workspace/triplet_session/output/mercado_livre/dataset/item__processed.csv", usecols=["item_id", "domain_id", "domain_idx"])#.head(10)
df_item.head()

,domain_id,item_id,domain_idx
0,MLM-INDIVIDUAL_HOUSES_FOR_SALE,111260,6107
1,MLM-VIDEO_GAMES,871377,7754
2,MLM-SKIRTS,490232,7275
3,MLM-GRAPHICS_CARDS,1150706,5892
4,MLM-NOTEBOOKS,934912,6707


In [3]:
domain_map = df_item[['item_id', 'domain_idx']].set_index("item_id").to_dict()["domain_idx"]
#domain_map

In [6]:
df_sub = pd.read_csv("/media/workspace/triplet_session/output/evaluation/MLEvaluationTask/results/SupervisedModelTraining____mars_gym_model_b____5d5ef06c6a_1581bc5508_sub_True/submission_SupervisedModelTraining____mars_gym_model_b____5d5ef06c6a_1581bc5508_sub_True.csv", header=None)
df_sub.head()

,0,1,2,3,4,5,6,7,8,9
0,1615991,1748830,1127740,1361154,1000062,1647194,345185,1628496,1843848,1589320
1,228737,645991,1832968,2035642,815151,490301,577128,1432809,1513922,1506406
2,1587422,1716388,1361154,290755,1906937,361733,671264,1839144,2032612,1158631
3,914466,516913,1596880,846812,1073489,1082484,1140653,804809,905801,519147
4,86082,1587422,1906937,725371,1080561,1843848,1643254,151458,573757,129029


In [7]:
df_sub.values

array([[1615991, 1748830, 1127740, ..., 1628496, 1843848, 1589320],
       [ 228737,  645991, 1832968, ..., 1432809, 1513922, 1506406],
       [1587422, 1716388, 1361154, ..., 1839144, 2032612, 1158631],
       ...,
       [1298130,  162136,  628598, ...,  797984,  746368, 1090382],
       [ 617673, 1583899,  633650, ..., 1235394, 1007974, 1150765],
       [1767506,  166738,  326171, ..., 1028116, 1524150,  596292]])

In [8]:
def _get_domain(arr, domain_map):
    return list(map(domain_map.get, arr))
    
with Pool(os.cpu_count()) as p:
    _map_domain = list(tqdm(
        p.map(functools.partial(_get_domain, domain_map=domain_map), df_sub.values),
        total=len(df_sub.values),
    ))  
    
_map_domain    

100%|██████████| 1000/1000 [00:00<00:00, 1558641.40it/s]


[[3513, 3513, 3513, 3513, 3905, 3513, 3472, 2939, 668, 3194],
 [2587, 3661, 3188, 2976, 7418, 3957, 3524, 2986, 1518, 1255],
 [2027, 3776, 3513, 815, 2027, 1966, 2027, 3513, 3524, 3671],
 [24, 3472, 3630, 3524, 560, 4195, 2718, 2572, 3376, 860],
 [2027, 2027, 2027, 3513, 2027, 668, 694, 2027, 2027, 2027],
 [2027, 3513, 680, 3630, 2553, 3513, 1999, 3776, 2027, 2027],
 [890, 1113, 2387, 890, 3671, 1985, 888, 2027, 1223, 3661],
 [678, 2402, 815, 3630, 3661, 3957, 888, 2027, 2190, 694],
 [5999, 5999, 6804, 6308, 5999, 5999, 4619, 7455, 6851, 5411],
 [1113, 1113, 1113, 1113, 1113, 1781, 3241, 3767, 766, 2906],
 [2027, 815, 3776, 3776, 3556, 3513, 3556, 815, 3556, 3513],
 [2027, 2027, 2027, 888, 815, 3513, 3964, 3524, 3986, 3513],
 [1653, 3966, 3672, 1653, 1792, 3651, 896, 2151, 896, 4997],
 [890, 888, 888, 888, 668, 669, 4192, 1730, 2015, 2027],
 [3549, 2781, 694, 680, 2027, 3513, 2602, 3850, 3524, 3567],
 [2874, 2874, 111, 2015, 2057, 2015, 3661, 694, 1292, 912],
 [3524, 2197, 1373, 815, 3

In [9]:
def _get_moda(arr):
    return stats.mode(arr)[0][0]

def _get_count_moda(arr):
    return stats.mode(arr)[1][0]/len(arr)

In [10]:
arr_moda = list(zip(list(df_sub.values), list(_map_domain),  list(map(_get_moda, _map_domain)), list(map(_get_count_moda, _map_domain))))
arr_moda

[(array([1615991, 1748830, 1127740, 1361154, 1000062, 1647194,  345185,
         1628496, 1843848, 1589320]),
  [3513, 3513, 3513, 3513, 3905, 3513, 3472, 2939, 668, 3194],
  3513,
  0.5),
 (array([ 228737,  645991, 1832968, 2035642,  815151,  490301,  577128,
         1432809, 1513922, 1506406]),
  [2587, 3661, 3188, 2976, 7418, 3957, 3524, 2986, 1518, 1255],
  1255,
  0.1),
 (array([1587422, 1716388, 1361154,  290755, 1906937,  361733,  671264,
         1839144, 2032612, 1158631]),
  [2027, 3776, 3513, 815, 2027, 1966, 2027, 3513, 3524, 3671],
  2027,
  0.3),
 (array([ 914466,  516913, 1596880,  846812, 1073489, 1082484, 1140653,
          804809,  905801,  519147]),
  [24, 3472, 3630, 3524, 560, 4195, 2718, 2572, 3376, 860],
  24,
  0.1),
 (array([  86082, 1587422, 1906937,  725371, 1080561, 1843848, 1643254,
          151458,  573757,  129029]),
  [2027, 2027, 2027, 3513, 2027, 668, 694, 2027, 2027, 2027],
  2027,
  0.7),
 (array([1906937,  725371,  956488,  992938,  986736, 115432

In [11]:
df_moda = pd.DataFrame(arr_moda, columns=["reclist", "domainlist", "domain_moda", "count"])
df_moda.head()

,reclist,domainlist,domain_moda,count
0,"[1615991, 1748830, 1127740, 1361154, 1000062, ...","[3513, 3513, 3513, 3513, 3905, 3513, 3472, 293...",3513,0.5
1,"[228737, 645991, 1832968, 2035642, 815151, 490...","[2587, 3661, 3188, 2976, 7418, 3957, 3524, 298...",1255,0.1
2,"[1587422, 1716388, 1361154, 290755, 1906937, 3...","[2027, 3776, 3513, 815, 2027, 1966, 2027, 3513...",2027,0.3
3,"[914466, 516913, 1596880, 846812, 1073489, 108...","[24, 3472, 3630, 3524, 560, 4195, 2718, 2572, ...",24,0.1
4,"[86082, 1587422, 1906937, 725371, 1080561, 184...","[2027, 2027, 2027, 3513, 2027, 668, 694, 2027,...",2027,0.7


In [12]:
def _create_relevance_list(sorted_actions, expected_action):
    s = []
    for i in range(len(sorted_actions)):
        if str(sorted_actions[i]) == str(expected_action):
            s.append(i + 0)
        else:
            s.append(i + 100)
    return s
    #return [0 if str(action) == str(expected_action) else 1 for action in sorted_actions]

df_moda['relevance_list'] = df_moda.apply(lambda row: 
                                          _create_relevance_list(row['domainlist'], row['domain_moda']),  
                                          axis=1)

In [13]:
df_moda.head()

,reclist,domainlist,domain_moda,count,relevance_list
0,"[1615991, 1748830, 1127740, 1361154, 1000062, ...","[3513, 3513, 3513, 3513, 3905, 3513, 3472, 293...",3513,0.5,"[0, 1, 2, 3, 104, 5, 106, 107, 108, 109]"
1,"[228737, 645991, 1832968, 2035642, 815151, 490...","[2587, 3661, 3188, 2976, 7418, 3957, 3524, 298...",1255,0.1,"[100, 101, 102, 103, 104, 105, 106, 107, 108, 9]"
2,"[1587422, 1716388, 1361154, 290755, 1906937, 3...","[2027, 3776, 3513, 815, 2027, 1966, 2027, 3513...",2027,0.3,"[0, 101, 102, 103, 4, 105, 6, 107, 108, 109]"
3,"[914466, 516913, 1596880, 846812, 1073489, 108...","[24, 3472, 3630, 3524, 560, 4195, 2718, 2572, ...",24,0.1,"[0, 101, 102, 103, 104, 105, 106, 107, 108, 109]"
4,"[86082, 1587422, 1906937, 725371, 1080561, 184...","[2027, 2027, 2027, 3513, 2027, 668, 694, 2027,...",2027,0.7,"[0, 1, 2, 103, 4, 105, 106, 7, 8, 9]"


In [23]:
def _sorte_by_domain_moda(reclist, relevance_list, percent, limit):
    if percent >= limit:
        return [x for _,x in sorted(zip(relevance_list, reclist), reverse=False)]
    else:
        return reclist
    
df_moda['reclist_2'] = df_moda.apply(lambda row: _sorte_by_domain_moda(
                                                row['reclist'], 
                                                row['relevance_list'], 
                                                row['count'], 0.2),  axis=1)    
df_moda['domainlist_2'] = df_moda.apply(lambda row: _sorte_by_domain_moda(
                                                row['domainlist'], 
                                                row['relevance_list'], 
                                                row['count'], 0.2),  axis=1)  

In [24]:
df_moda.head()

,reclist,domainlist,domain_moda,count,relevance_list,reclist_2,domainlist_2
0,"[1615991, 1748830, 1127740, 1361154, 1000062, ...","[3513, 3513, 3513, 3513, 3905, 3513, 3472, 293...",3513,0.5,"[0, 1, 2, 3, 104, 5, 106, 107, 108, 109]","[1615991, 1748830, 1127740, 1361154, 1647194, ...","[3513, 3513, 3513, 3513, 3513, 3905, 3472, 293..."
1,"[228737, 645991, 1832968, 2035642, 815151, 490...","[2587, 3661, 3188, 2976, 7418, 3957, 3524, 298...",1255,0.1,"[100, 101, 102, 103, 104, 105, 106, 107, 108, 9]","[228737, 645991, 1832968, 2035642, 815151, 490...","[2587, 3661, 3188, 2976, 7418, 3957, 3524, 298..."
2,"[1587422, 1716388, 1361154, 290755, 1906937, 3...","[2027, 3776, 3513, 815, 2027, 1966, 2027, 3513...",2027,0.3,"[0, 101, 102, 103, 4, 105, 6, 107, 108, 109]","[1587422, 1906937, 671264, 1716388, 1361154, 2...","[2027, 2027, 2027, 3776, 3513, 815, 1966, 3513..."
3,"[914466, 516913, 1596880, 846812, 1073489, 108...","[24, 3472, 3630, 3524, 560, 4195, 2718, 2572, ...",24,0.1,"[0, 101, 102, 103, 104, 105, 106, 107, 108, 109]","[914466, 516913, 1596880, 846812, 1073489, 108...","[24, 3472, 3630, 3524, 560, 4195, 2718, 2572, ..."
4,"[86082, 1587422, 1906937, 725371, 1080561, 184...","[2027, 2027, 2027, 3513, 2027, 668, 694, 2027,...",2027,0.7,"[0, 1, 2, 103, 4, 105, 106, 7, 8, 9]","[86082, 1587422, 1906937, 1080561, 151458, 573...","[2027, 2027, 2027, 2027, 2027, 2027, 2027, 351..."


In [16]:
df_moda.iloc[2]

reclist           [1587422, 1716388, 1361154, 290755, 1906937, 3...
domainlist        [2027, 3776, 3513, 815, 2027, 1966, 2027, 3513...
domain_moda                                                    2027
count                                                           0.3
relevance_list         [0, 101, 102, 103, 4, 105, 6, 107, 108, 109]
reclist_2         [1587422, 1906937, 671264, 1716388, 1361154, 2...
Name: 2, dtype: object

In [17]:
df_moda.iloc[2].reclist

array([1587422, 1716388, 1361154,  290755, 1906937,  361733,  671264,
       1839144, 2032612, 1158631])

In [18]:
df_moda.iloc[2].domainlist

[2027, 3776, 3513, 815, 2027, 1966, 2027, 3513, 3524, 3671]

In [19]:
df_moda.iloc[2].reclist_2

[1587422,
 1906937,
 671264,
 1716388,
 1361154,
 290755,
 361733,
 1839144,
 2032612,
 1158631]

In [20]:
a = df_moda.reclist_2.apply(lambda x: np.array(x[:10])).values
a[0]

array([1615991, 1748830, 1127740, 1361154, 1647194, 1000062,  345185,
       1628496, 1843848, 1589320])

In [21]:
def _relevance_list_ml(reclist, domainlist, item_id, domain_id):
    s = []
    for i in range(len(reclist)):
        if str(reclist[i]) == str(item_id):
            value = 12
        elif str(domainlist[i]) == str(domain_id):
            value = 1
        else:
            value = 0
        s.append(value)
    return s    

In [27]:
_relevance_list_ml(df_moda.iloc[2].reclist_2, df_moda.iloc[2].domainlist_2, 1906937, 2027)

[1, 12, 1, 0, 0, 0, 0, 0, 0, 0]